In [1]:
# Various pip
!pip install tensorflow
!pip install transformers
!pip install datasets

In [2]:
# Dataframe management
import pandas as pd

# Data manipulation
import numpy as np
import random

# NNs
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Transformers
import transformers

# Convert to HuggingFace dataset
import pyarrow as pa
from datasets import Dataset
import datasets

# Sklearn for train test split
from sklearn.model_selection import train_test_split

In [3]:
# Get dataset
df_en = pd.read_csv('mainDataEN.csv')

df_it = pd.read_csv('haspeede2_dev_taskAB.tsv', sep='\t')

In [4]:
def is_hate_speech(value):
  if value['class'] == 2:
    return 0 #no hate speech
  else:
     return 1 #hate speech

df_en['hate_speech'] = df_en.apply(is_hate_speech, axis=1)

df_en = df_en.drop(['Unnamed: 0','count', 'offensive_language', 'neither', 'class'], axis=1)

df_en.head()

,hate_speech,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
df_it = df_it.drop(['id','stereotype'], axis=1)
df_it

,text,hs
0,"È terrorismo anche questo, per mettere in uno ...",0
1,@user @user infatti finché ci hanno guadagnato...,0
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
3,"@user ad uno ad uno, perché quando i migranti ...",0
4,Il divertimento del giorno? Trovare i patrioti...,0
...,...,...
6832,Gli stati nazionali devono essere pronti a rin...,0
6833,Il ministro dell'interno della Germania #Horst...,0
6834,#Salvini: In Italia troppi si sono montati la ...,0
6835,@user @user Chi giubila in buona fede non ha c...,0


In [6]:
train_en, test_en = train_test_split(df_en, test_size=0.2)

X_train = train_en['tweet'].reset_index(drop=True)
Y_train = train_en['hate_speech'].reset_index(drop=True)

X_test = test_en['tweet'].reset_index(drop=True)
Y_test = test_en['hate_speech'].reset_index(drop=True)

X_it = df_it['text ']
Y_it = df_it['hs']

In [7]:
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec -O wiki.en.vec
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec -O wiki.it.vec

--2023-06-29 10:24:51--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.160.80, 99.84.160.61, 99.84.160.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.160.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628614720 (599M) [text/plain]
Saving to: ‘wiki.en.vec’

wiki.en.vec         100%[===================>] 599.49M   126MB/s    in 4.2s    

2023-06-29 10:24:56 (143 MB/s) - ‘wiki.en.vec’ saved [628614720/628614720]

--2023-06-29 10:24:56--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.84.160.80, 99.84.160.61, 99.84.160.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.84.160.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628718043 (600M) [text/plain]
Saving to: ‘wiki.it.vec’

wiki.it.vec         100%[============

In [8]:
import numpy as np

def load_vectors(embedding_file_path):
    print("Loading vectors from", embedding_file_path)
    embeddings = []
    word2id = {}
    with open(embedding_file_path, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            word, emb = line.rstrip().split(' ', 1)
            emb = np.fromstring(emb, sep=' ')
            assert word not in word2id, 'word found twice'
            embeddings.append(emb)
            word2id[word] = len(word2id)

    embeddings = np.vstack(embeddings)
    return embeddings, word2id

# Produce the:
#   - embeddings_en → list of the embeddings of the words in wiki
#   - embedding_word2id_en → dictionary with each word and relative index in the embedding list
embeddings_en, embedding_word2id_en = load_vectors("wiki.en.vec")
embeddings_it, embedding_word2id_it = load_vectors("wiki.it.vec")


Loading vectors from wiki.en.vec
Loading vectors from wiki.it.vec


In [9]:
embeddings_en

array([[-0.0112864 , -0.00206967, -0.0515041 , ...,  0.0435525 ,
        -0.00774608,  0.0724234 ],
       [-0.0469256 , -0.00058526, -0.0750844 , ...,  0.0267626 ,
        -0.0513555 ,  0.0166127 ],
       [-0.0324474 , -0.0462027 , -0.00872643, ...,  0.0826505 ,
        -0.0649553 ,  0.0175795 ],
       ...,
       [-0.0608303 , -0.0172956 , -0.101635  , ...,  0.102298  ,
         0.0479459 ,  0.0387411 ],
       [-0.0867473 , -0.00469176, -0.104337  , ..., -0.00478122,
         0.130826  , -0.0207968 ],
       [-0.0119003 , -0.0123243 , -0.065963  , ...,  0.0184328 ,
        -0.0295059 , -0.0160871 ]])

In [10]:
embedding_word2id_en

{',': 0,
 '.': 1,
 'the': 2,
 '</s>': 3,
 'of': 4,
 '-': 5,
 'in': 6,
 'and': 7,
 "'": 8,
 ')': 9,
 '(': 10,
 'to': 11,
 'a': 12,
 'is': 13,
 'was': 14,
 'on': 15,
 's': 16,
 'for': 17,
 'as': 18,
 'by': 19,
 'that': 20,
 'it': 21,
 'with': 22,
 'from': 23,
 'at': 24,
 'he': 25,
 'this': 26,
 'be': 27,
 'i': 28,
 'an': 29,
 'utc': 30,
 'his': 31,
 'not': 32,
 '–': 33,
 'are': 34,
 'or': 35,
 'talk': 36,
 'which': 37,
 'also': 38,
 'has': 39,
 'were': 40,
 'but': 41,
 'have': 42,
 '#': 43,
 'one': 44,
 'rd': 45,
 'new': 46,
 'first': 47,
 'page': 48,
 'no': 49,
 'you': 50,
 'they': 51,
 'had': 52,
 'article': 53,
 't': 54,
 'who': 55,
 '?': 56,
 'all': 57,
 'their': 58,
 'there': 59,
 'been': 60,
 'made': 61,
 'its': 62,
 'people': 63,
 'may': 64,
 'after': 65,
 '%': 66,
 'other': 67,
 'should': 68,
 'two': 69,
 'score': 70,
 'her': 71,
 'can': 72,
 'would': 73,
 'more': 74,
 'if': 75,
 'she': 76,
 'about': 77,
 'when': 78,
 'time': 79,
 'team': 80,
 'american': 81,
 'such': 82,
 'th': 

In [11]:
# Deinfe some variables
VOCABULARY_SIZE = 10000
EMBEDDING_DIM = 300
SEQ_LENGTH = 300

INDEX_FROM = 3

In [12]:
X_train = ['<START> ' + x for x in X_train]
X_test= ['<START> ' + x for x in X_test]

In [13]:
# create the tokenizer
t = Tokenizer(oov_token="<unk>", num_words=VOCABULARY_SIZE+INDEX_FROM-1, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

# fit the tokenizer on the documents
t.fit_on_texts(X_train)

t.word_index['<pad>'] = 0
t.index_word[0] = '<pad>'

# summarize what was learned
#print(t.word_counts)
#print(t.document_count)
print(t.word_index)
print(t.word_docs)

# integer encode documents
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

{'<unk>': 1, '<start>': 2, 'a': 3, 'bitch': 4, 'i': 5, 'rt': 6, 'the': 7, 'you': 8, 'to': 9, 'and': 10, 'my': 11, 'that': 12, 't': 13, '128514': 14, 'bitches': 15, 'in': 16, 'co': 17, 'http': 18, 'is': 19, 'like': 20, 'me': 21, 'of': 22, 'on': 23, 'hoes': 24, 'be': 25, 'this': 26, 'pussy': 27, 'for': 28, 'it': 29, 'hoe': 30, 'with': 31, '8220': 32, '8221': 33, "i'm": 34, 'ass': 35, 'your': 36, 'up': 37, 'all': 38, 'if': 39, 'but': 40, "don't": 41, 'just': 42, 'get': 43, 'fuck': 44, 'so': 45, 'they': 46, 'when': 47, 'got': 48, 'no': 49, 'these': 50, 'shit': 51, 'u': 52, '8230': 53, 'nigga': 54, 'not': 55, 'she': 56, 'trash': 57, 'her': 58, 'are': 59, 'was': 60, 'at': 61, 'lol': 62, 'out': 63, 'have': 64, 'about': 65, 'amp': 66, 'he': 67, 'what': 68, "ain't": 69, 'some': 70, 'do': 71, 'can': 72, 'niggas': 73, 'know': 74, 'we': 75, 'how': 76, 'them': 77, 'love': 78, 'one': 79, "it's": 80, 'or': 81, 'who': 82, 'as': 83, 'fucking': 84, '128557': 85, 'go': 86, 'now': 87, 'yo': 88, 'from': 89

In [14]:
X_train = tf.keras.utils.pad_sequences(X_train, maxlen=SEQ_LENGTH)
X_test = tf.keras.utils.pad_sequences(X_test, maxlen=SEQ_LENGTH)

In [15]:
len(X_train[76])

300

In [16]:
def create_embedding_matrix(target_word2id, embedding_word2id, embeddings, num_rows, num_columns):
    embedding_matrix = np.zeros((num_rows, num_columns))
    for word, i in target_word2id.items():
        if i >= num_rows:
            break
        if word in embedding_word2id:
            embedding_matrix[i] = embeddings[embedding_word2id[word]]
    return embedding_matrix

In [17]:
word2id_en = t.word_index

embedding_matrix_en = create_embedding_matrix(word2id_en, embedding_word2id_en, embeddings_en, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)

In [18]:
embedding_matrix_en

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.0757012 ,  0.00026433,  0.0048104 , ...,  0.0172276 ,
        -0.00987242,  0.0452261 ],
       [-0.0778776 ,  0.0426349 , -0.0539983 , ...,  0.0202902 ,
        -0.0505175 ,  0.0721214 ]])

In [19]:
# Model
model = Sequential()
# ------
model.add(keras.layers.Embedding(VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM, input_length=SEQ_LENGTH, weights=[embedding_matrix_en], trainable=False))
# ------
model.add(keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(250, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3000600   
                                                                 
 conv1d (Conv1D)             (None, 300, 32)           28832     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 150, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 4800)              0         
                                                                 
 dense (Dense)               (None, 250)               1200250   
                                                                 
 dense_1 (Dense)             (None, 1)                 251       
                                                        

In [20]:
Y_train

0        1
1        1
2        0
3        1
4        1
        ..
19821    1
19822    1
19823    1
19824    1
19825    1
Name: hate_speech, Length: 19826, dtype: int64

In [21]:
NUM_EPOCHS=20
BATCH_SIZE=64

earlystop = EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
          epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[earlystop])

# Final evaluation of the model
scores = model.evaluate(X_test, Y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
310/310 [==============================] - 72s 226ms/step - loss: 0.2255 - accuracy: 0.9029 - val_loss: 0.1473 - val_accuracy: 0.9391
Epoch 2/20
310/310 [==============================] - 53s 171ms/step - loss: 0.1363 - accuracy: 0.9429 - val_loss: 0.1333 - val_accuracy: 0.9447
Epoch 3/20
310/310 [==============================] - 55s 177ms/step - loss: 0.1149 - accuracy: 0.9543 - val_loss: 0.1285 - val_accuracy: 0.9461
Epoch 4/20
310/310 [==============================] - 65s 210ms/step - loss: 0.0941 - accuracy: 0.9639 - val_loss: 0.1350 - val_accuracy: 0.9429
Epoch 5/20
155/155 [==============================] - 4s 24ms/step - loss: 0.1785 - accuracy: 0.9282
Accuracy: 92.82%


In [22]:


model.predict(X_test[13])

ValueError: ignored